In [1]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
import torch.nn.functional as F

import seaborn as sns
import matplotlib.pyplot as plt
import os
from datetime import date
import datetime

C:\Users\ladwi\anaconda3\envs\deeplearning\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Generating the training data for the Heat and Diffusion Model

In [2]:
data_dir = "./LakePIAB/"
depth_steps = 25 * 2

print(os.getcwd())

C:\Users\ladwi\Documents\Projects\R\LakePIAB\MCL\01_data_processing


In [3]:
meterological_data_df = pd.read_csv("./../../output/py_meteorology_input.csv")
meterological_data_df = meterological_data_df[1:] # considering everything from 2nd time step

num_time_steps = meterological_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
meterological_data_df = pd.DataFrame(np.repeat(meterological_data_df.values, depth_steps, axis=0), columns=meterological_data_df.columns)
meterological_data_df = pd.concat([depth_df, meterological_data_df], ignore_index=False, axis=1)
meterological_data_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,icemovAvg,density_snow,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,250.0,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,-0.303065,250.0,0.283046,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,-0.303065,250.0,0.283046,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,-0.303065,250.0,0.283046,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,-0.303065,250.0,0.283046,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065


In [4]:
input_temp_df = pd.read_csv("./../../output/py_temp_total05.csv")

flattened_inp_temp = input_temp_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
flattened_out_temp = input_temp_df.iloc[1:,1:].to_numpy().flatten() #this iloc is to remove the time column
time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values
data = {'time':time_stamp, 'input_temp':flattened_inp_temp, 'temp_total05':flattened_out_temp, 'depth':depth_list}
input_temp_df = pd.DataFrame(data=data)
input_temp_df 

,time,input_temp,temp_total05,depth
0,2015-12-30 01:00:00,-0.000127,-0.115875,1
1,2015-12-30 01:00:00,0.814294,-0.019017,2
2,2015-12-30 01:00:00,1.500005,0.046871,3
3,2015-12-30 01:00:00,1.500005,0.799300,4
4,2015-12-30 01:00:00,1.500005,1.381346,5
...,...,...,...,...
1751945,2019-12-28 23:00:00,4.155676,4.155743,46
1751946,2019-12-28 23:00:00,4.250646,4.250735,47
1751947,2019-12-28 23:00:00,4.347803,4.347929,48
1751948,2019-12-28 23:00:00,4.438366,4.438489,49


In [5]:
depth_list.shape

(1751950,)

In [6]:
final_df = meterological_data_df.merge(input_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,ice_prior,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,-0.000127,-0.115875
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,0.814294,-0.019017
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,0.046871
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,0.799300
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,1.381346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.283046,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.155676,4.155743
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.283046,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.250646,4.250735
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.283046,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.347803,4.347929
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.283046,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.438366,4.438489


In [7]:
buoyancy_data_df = pd.read_csv("./../../output/py_buoyancy.csv")
#Removing Nans
#buoyancy_data_df['n2S-2_1'] = buoyancy_data_df['n2S-2_2']
#buoyancy_data_df['n2S-2_25'] = buoyancy_data_df['n2S-2_24']
buoyancy_data_df['24'] = buoyancy_data_df['23']

buoyancy_data_df = buoyancy_data_df[1:] # considering everything from 2nd time step

flattened_buoy = buoyancy_data_df.iloc[:,1:].to_numpy().flatten()
time_stamp = buoyancy_data_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'buoyancy':flattened_buoy, 'depth':depth_list}
buoyancy_data_df = pd.DataFrame(data=data)

buoyancy_data_df

,time,buoyancy,depth
0,2015-12-30 01:00:00,0.000827,1
1,2015-12-30 01:00:00,0.000640,2
2,2015-12-30 01:00:00,0.000000,3
3,2015-12-30 01:00:00,0.000218,4
4,2015-12-30 01:00:00,0.000154,5
...,...,...,...
1751945,2019-12-28 23:00:00,0.000007,46
1751946,2019-12-28 23:00:00,0.000010,47
1751947,2019-12-28 23:00:00,0.000012,48
1751948,2019-12-28 23:00:00,0.000016,49


In [8]:
final_df = final_df.merge(buoyancy_data_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,snow_prior,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,-0.000127,-0.115875,0.000827
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,0.814294,-0.019017,0.000640
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,0.046871,0.000000
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,0.799300,0.000218
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,1.381346,0.000154
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.155676,4.155743,0.000007
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.250646,4.250735,0.000010
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.347803,4.347929,0.000012
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.025199,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.438366,4.438489,0.000016


In [9]:
out_diffusivity_df = pd.read_csv("./../../output/py_diff.csv")

out_diffusivity_df = out_diffusivity_df[1:] # considering everything from 2nd time step

flattened_diff = out_diffusivity_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_diffusivity_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'diffusivity':flattened_diff, 'depth':depth_list}
out_diffusivity_df = pd.DataFrame(data=data)

out_diffusivity_df

,time,diffusivity,depth
0,2015-12-30 01:00:00,1.425118e-02,1
1,2015-12-30 01:00:00,1.425118e-02,2
2,2015-12-30 01:00:00,5.296182e-03,3
3,2015-12-30 01:00:00,2.857333e-04,4
4,2015-12-30 01:00:00,1.784540e-05,5
...,...,...,...
1751945,2019-12-28 23:00:00,2.800001e-07,46
1751946,2019-12-28 23:00:00,2.800001e-07,47
1751947,2019-12-28 23:00:00,2.800000e-07,48
1751948,2019-12-28 23:00:00,2.800000e-07,49


In [10]:
final_df = final_df.merge(out_diffusivity_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,snowice_prior,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.385235,-0.000127,-0.115875,0.000827,1.425118e-02
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.385235,0.814294,-0.019017,0.000640,1.425118e-02
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,0.046871,0.000000,5.296182e-03
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,0.799300,0.000218,2.857333e-04
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.0,250.0,1.0,0.0,0.8,5.385235,1.500005,1.381346,0.000154,1.784540e-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.155676,4.155743,0.000007,2.800001e-07
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.250646,4.250735,0.000010,2.800001e-07
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.347803,4.347929,0.000012,2.800000e-07
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.044785,250.0,0.749743,1.0,0.8,-0.303065,4.438366,4.438489,0.000016,2.800000e-07


In [11]:
out_temp_df = pd.read_csv("./../../output/py_temp_heat01.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_heat01':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_heat01,depth
0,2015-12-30 01:00:00,-0.115875,1
1,2015-12-30 01:00:00,-0.019017,2
2,2015-12-30 01:00:00,0.046871,3
3,2015-12-30 01:00:00,0.799300,4
4,2015-12-30 01:00:00,1.381346,5
...,...,...,...
1751945,2019-12-28 23:00:00,4.155743,46
1751946,2019-12-28 23:00:00,4.250735,47
1751947,2019-12-28 23:00:00,4.347929,48
1751948,2019-12-28 23:00:00,4.438489,49


In [12]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,rho_snow_prior,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.385235,-0.000127,-0.115875,0.000827,1.425118e-02,-0.115875
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.385235,0.814294,-0.019017,0.000640,1.425118e-02,-0.019017
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.385235,1.500005,0.046871,0.000000,5.296182e-03,0.046871
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.385235,1.500005,0.799300,0.000218,2.857333e-04,0.799300
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,250.0,1.0,0.0,0.8,5.385235,1.500005,1.381346,0.000154,1.784540e-05,1.381346
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,250.0,0.749743,1.0,0.8,-0.303065,4.155676,4.155743,0.000007,2.800001e-07,4.155743
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,250.0,0.749743,1.0,0.8,-0.303065,4.250646,4.250735,0.000010,2.800001e-07,4.250735
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,250.0,0.749743,1.0,0.8,-0.303065,4.347803,4.347929,0.000012,2.800000e-07,4.347929
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,250.0,0.749743,1.0,0.8,-0.303065,4.438366,4.438489,0.000016,2.800000e-07,4.438489


In [13]:
out_temp_df = pd.read_csv("./../../output/py_temp_diff02.csv")

out_temp_df = out_temp_df[1:] # considering everything from 2nd time step

flattened_temp = out_temp_df.iloc[:,1:].to_numpy().flatten()
time_stamp = out_temp_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_diff02':flattened_temp, 'depth':depth_list}
out_temp_df = pd.DataFrame(data=data)

out_temp_df

,time,temp_diff02,depth
0,2015-12-30 01:00:00,-0.115875,1
1,2015-12-30 01:00:00,0.470220,2
2,2015-12-30 01:00:00,1.092054,3
3,2015-12-30 01:00:00,1.054755,4
4,2015-12-30 01:00:00,1.312011,5
...,...,...,...
1751945,2019-12-28 23:00:00,4.155746,46
1751946,2019-12-28 23:00:00,4.250744,47
1751947,2019-12-28 23:00:00,4.347902,48
1751948,2019-12-28 23:00:00,4.438525,49


In [14]:
final_df = final_df.merge(out_temp_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,IceSnowAttCoeff_prior,iceFlag_prior,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.385235,-0.000127,-0.115875,0.000827,1.425118e-02,-0.115875,-0.115875
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.385235,0.814294,-0.019017,0.000640,1.425118e-02,-0.019017,0.470220
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.385235,1.500005,0.046871,0.000000,5.296182e-03,0.046871,1.092054
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.385235,1.500005,0.799300,0.000218,2.857333e-04,0.799300,1.054755
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.0,0.0,0.8,5.385235,1.500005,1.381346,0.000154,1.784540e-05,1.381346,1.312011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.749743,1.0,0.8,-0.303065,4.155676,4.155743,0.000007,2.800001e-07,4.155743,4.155746
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.749743,1.0,0.8,-0.303065,4.250646,4.250735,0.000010,2.800001e-07,4.250735,4.250744
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.749743,1.0,0.8,-0.303065,4.347803,4.347929,0.000012,2.800000e-07,4.347929,4.347902
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.749743,1.0,0.8,-0.303065,4.438366,4.438489,0.000016,2.800000e-07,4.438489,4.438525


In [15]:
datetime_list =[datetime.datetime.strptime(date, '%Y-%m-%d %H:%M:%S') for date in final_df['time']]
day_of_year_list = [t.timetuple().tm_yday for t in datetime_list]
time_of_day_list = [t.hour for t in datetime_list]

In [16]:
final_df['day_of_year']=day_of_year_list
final_df['time_of_day']=time_of_day_list

In [17]:
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,dt_iceon_avg_prior,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.8,5.385235,-0.000127,-0.115875,0.000827,1.425118e-02,-0.115875,-0.115875,364,1
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.8,5.385235,0.814294,-0.019017,0.000640,1.425118e-02,-0.019017,0.470220,364,1
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.8,5.385235,1.500005,0.046871,0.000000,5.296182e-03,0.046871,1.092054,364,1
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.8,5.385235,1.500005,0.799300,0.000218,2.857333e-04,0.799300,1.054755,364,1
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.8,5.385235,1.500005,1.381346,0.000154,1.784540e-05,1.381346,1.312011,364,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.8,-0.303065,4.155676,4.155743,0.000007,2.800001e-07,4.155743,4.155746,362,23
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.8,-0.303065,4.250646,4.250735,0.000010,2.800001e-07,4.250735,4.250744,362,23
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.8,-0.303065,4.347803,4.347929,0.000012,2.800000e-07,4.347929,4.347902,362,23
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.8,-0.303065,4.438366,4.438489,0.000016,2.800000e-07,4.438489,4.438525,362,23


In [18]:
temp_mix03_df = pd.read_csv("./../../output/py_temp_mix03.csv")

temp_mix03_df = temp_mix03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_mix03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_mix03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_mix03':flattened_temp, 'depth':depth_list}
temp_mix03_df = pd.DataFrame(data=data)

temp_mix03_df

,time,temp_mix03,depth
0,2015-12-30 01:00:00,-0.115875,1
1,2015-12-30 01:00:00,0.470220,2
2,2015-12-30 01:00:00,1.092054,3
3,2015-12-30 01:00:00,1.054755,4
4,2015-12-30 01:00:00,1.312011,5
...,...,...,...
1751945,2019-12-28 23:00:00,4.155746,46
1751946,2019-12-28 23:00:00,4.250744,47
1751947,2019-12-28 23:00:00,4.347902,48
1751948,2019-12-28 23:00:00,4.438525,49


In [19]:
final_df = final_df.merge(temp_mix03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,icemovAvg_prior,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,-0.000127,-0.115875,0.000827,1.425118e-02,-0.115875,-0.115875,364,1,-0.115875
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,0.814294,-0.019017,0.000640,1.425118e-02,-0.019017,0.470220,364,1,0.470220
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,1.500005,0.046871,0.000000,5.296182e-03,0.046871,1.092054,364,1,1.092054
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,1.500005,0.799300,0.000218,2.857333e-04,0.799300,1.054755,364,1,1.054755
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.385235,1.500005,1.381346,0.000154,1.784540e-05,1.381346,1.312011,364,1,1.312011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,-0.303065,4.155676,4.155743,0.000007,2.800001e-07,4.155743,4.155746,362,23,4.155746
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,-0.303065,4.250646,4.250735,0.000010,2.800001e-07,4.250735,4.250744,362,23,4.250744
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,-0.303065,4.347803,4.347929,0.000012,2.800000e-07,4.347929,4.347902,362,23,4.347902
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,-0.303065,4.438366,4.438489,0.000016,2.800000e-07,4.438489,4.438525,362,23,4.438525


In [20]:
temp_conv03_df = pd.read_csv("./../../output/py_temp_conv04.csv")

temp_conv03_df = temp_conv03_df[1:] # considering everything from 2nd time step

flattened_temp = temp_conv03_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_conv03_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_conv04':flattened_temp, 'depth':depth_list}
temp_conv03_df = pd.DataFrame(data=data)

temp_conv03_df

,time,temp_conv04,depth
0,2015-12-30 01:00:00,-0.115875,1
1,2015-12-30 01:00:00,0.470220,2
2,2015-12-30 01:00:00,1.073671,3
3,2015-12-30 01:00:00,1.073671,4
4,2015-12-30 01:00:00,1.312011,5
...,...,...,...
1751945,2019-12-28 23:00:00,4.155746,46
1751946,2019-12-28 23:00:00,4.250744,47
1751947,2019-12-28 23:00:00,4.347902,48
1751948,2019-12-28 23:00:00,4.438525,49


In [21]:
final_df = final_df.merge(temp_conv03_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,input_temp,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,-0.000127,-0.115875,0.000827,1.425118e-02,-0.115875,-0.115875,364,1,-0.115875,-0.115875
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.814294,-0.019017,0.000640,1.425118e-02,-0.019017,0.470220,364,1,0.470220,0.470220
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.500005,0.046871,0.000000,5.296182e-03,0.046871,1.092054,364,1,1.092054,1.073671
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.500005,0.799300,0.000218,2.857333e-04,0.799300,1.054755,364,1,1.054755,1.073671
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.500005,1.381346,0.000154,1.784540e-05,1.381346,1.312011,364,1,1.312011,1.312011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,4.155676,4.155743,0.000007,2.800001e-07,4.155743,4.155746,362,23,4.155746,4.155746
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,4.250646,4.250735,0.000010,2.800001e-07,4.250735,4.250744,362,23,4.250744,4.250744
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,4.347803,4.347929,0.000012,2.800000e-07,4.347929,4.347902,362,23,4.347902,4.347902
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,4.438366,4.438489,0.000016,2.800000e-07,4.438489,4.438525,362,23,4.438525,4.438525


In [22]:
temp_initial00_df = pd.read_csv("./../../output/py_temp_initial00.csv")

temp_initial00_df = temp_initial00_df[1:] # considering everything from 2nd time step

flattened_temp = temp_initial00_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_initial00_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'temp_initial00':flattened_temp, 'depth':depth_list}
temp_initial00_df = pd.DataFrame(data=data)

temp_initial00_df

,time,temp_initial00,depth
0,2015-12-30 01:00:00,-0.019025,1
1,2015-12-30 01:00:00,-0.019025,2
2,2015-12-30 01:00:00,0.046866,3
3,2015-12-30 01:00:00,0.799295,4
4,2015-12-30 01:00:00,1.381341,5
...,...,...,...
1751945,2019-12-28 23:00:00,4.155679,46
1751946,2019-12-28 23:00:00,4.250654,47
1751947,2019-12-28 23:00:00,4.347777,48
1751948,2019-12-28 23:00:00,4.438403,49


In [23]:
final_df = final_df.merge(temp_initial00_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,temp_total05,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,-0.115875,0.000827,1.425118e-02,-0.115875,-0.115875,364,1,-0.115875,-0.115875,-0.019025
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,-0.019017,0.000640,1.425118e-02,-0.019017,0.470220,364,1,0.470220,0.470220,-0.019025
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.046871,0.000000,5.296182e-03,0.046871,1.092054,364,1,1.092054,1.073671,0.046866
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.799300,0.000218,2.857333e-04,0.799300,1.054755,364,1,1.054755,1.073671,0.799295
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.381346,0.000154,1.784540e-05,1.381346,1.312011,364,1,1.312011,1.312011,1.381341
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,4.155743,0.000007,2.800001e-07,4.155743,4.155746,362,23,4.155746,4.155746,4.155679
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,4.250735,0.000010,2.800001e-07,4.250735,4.250744,362,23,4.250744,4.250744,4.250654
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,4.347929,0.000012,2.800000e-07,4.347929,4.347902,362,23,4.347902,4.347902,4.347777
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,4.438489,0.000016,2.800000e-07,4.438489,4.438525,362,23,4.438525,4.438525,4.438403


In [24]:
temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

temp_obs_df = temp_obs_df[1:] # considering everything from 2nd time step

flattened_temp = temp_obs_df.iloc[:,1:].to_numpy().flatten()
time_stamp = temp_obs_df['time'].repeat(depth_steps).values

data = {'time':time_stamp, 'obs_temp':flattened_temp, 'depth':depth_list}

print(data)
print(len(data['time']))
print(len(data['obs_temp']))
print(len(data['depth']))

temp_obs_df = pd.DataFrame(data=data)

temp_obs_df

print(flattened_temp.shape)

print(time_stamp.shape)

print(depth_list.shape)

{'time': array(['2015-12-30 01:00:00', '2015-12-30 01:00:00',
       '2015-12-30 01:00:00', ..., '2019-12-28 23:00:00',
       '2019-12-28 23:00:00', '2019-12-28 23:00:00'], dtype=object), 'obs_temp': array([ 5.55809349,  4.87020349,  5.10692829, ...,  5.53209381,
        7.39132851, 11.0079646 ]), 'depth': array([ 1,  2,  3, ..., 48, 49, 50])}
1751950
1751950
1751950
(1751950,)
(1751950,)
(1751950,)


In [25]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,buoyancy,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.000827,1.425118e-02,-0.115875,-0.115875,364,1,-0.115875,-0.115875,-0.019025,5.558093
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.000640,1.425118e-02,-0.019017,0.470220,364,1,0.470220,0.470220,-0.019025,4.870203
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.000000,5.296182e-03,0.046871,1.092054,364,1,1.092054,1.073671,0.046866,5.106928
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.000218,2.857333e-04,0.799300,1.054755,364,1,1.054755,1.073671,0.799295,5.106928
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,0.000154,1.784540e-05,1.381346,1.312011,364,1,1.312011,1.312011,1.381341,5.106928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1751945,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.000007,2.800001e-07,4.155743,4.155746,362,23,4.155746,4.155746,4.155679,4.045759
1751946,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.000010,2.800001e-07,4.250735,4.250744,362,23,4.250744,4.250744,4.250654,4.435799
1751947,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.000012,2.800000e-07,4.347929,4.347902,362,23,4.347902,4.347902,4.347777,5.532094
1751948,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,0.000016,2.800000e-07,4.438489,4.438525,362,23,4.438525,4.438525,4.438403,7.391329


In [26]:

#temp_obs_df2 = pd.read_csv("./../output/observed_temp.csv")
#temp_obs_df = pd.read_csv("./../output/observed_temp.csv")


#print(flattened_inp_temp_obs.shape)

#temp_obs_df2 = temp_obs_df2[:-1] # considering everything from 2nd time step

#flattened_inp_temp_obs = temp_obs_df2.iloc[:,1:].to_numpy().flatten()

#time_stamp = input_temp_df['time'][1:].repeat(depth_steps).values

#data = {'time':time_stamp, 'obs_input':flattened_inp_temp_obs, 'depth':depth_list}
#temp_obs_df2 = pd.DataFrame(data=data)

#temp_obs_df2


temp_obs_df = pd.read_csv("./../../output/py_observed_temp.csv")

flattened_inp_temp = temp_obs_df.iloc[:-1,1:].to_numpy().flatten() #this iloc is to remove the time column
print(flattened_inp_temp.shape)


time_stamp = temp_obs_df['time'][1:].repeat(depth_steps).values
print(time_stamp.shape)

print(depth_list.shape)
data = {'time':time_stamp, 'input_obs':flattened_inp_temp,'depth':depth_list}
temp_obs_df = pd.DataFrame(data=data)
temp_obs_df 

(1751950,)
(1751950,)
(1751950,)


,time,input_obs,depth
0,2015-12-30 01:00:00,5.560076,1
1,2015-12-30 01:00:00,4.876163,2
2,2015-12-30 01:00:00,5.112209,3
3,2015-12-30 01:00:00,5.112209,4
4,2015-12-30 01:00:00,5.112209,5
...,...,...,...
1751945,2019-12-28 23:00:00,4.047768,46
1751946,2019-12-28 23:00:00,4.438573,47
1751947,2019-12-28 23:00:00,5.534868,48
1751948,2019-12-28 23:00:00,7.391820,49


In [27]:
final_df = final_df.merge(temp_obs_df, how='inner', on=['time','depth'])
final_df

,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,diffusivity,temp_heat01,temp_diff02,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.425118e-02,-0.115875,-0.115875,364,1,-0.115875,-0.115875,-0.019025,5.558093,5.560076
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.425118e-02,-0.019017,0.470220,364,1,0.470220,0.470220,-0.019025,4.870203,4.876163
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,5.296182e-03,0.046871,1.092054,364,1,1.092054,1.073671,0.046866,5.106928,5.112209
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,2.857333e-04,0.799300,1.054755,364,1,1.054755,1.073671,0.799295,5.106928,5.112209
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,1.784540e-05,1.381346,1.312011,364,1,1.312011,1.312011,1.381341,5.106928,5.112209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752345,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,2.800001e-07,4.155743,4.155746,362,23,4.155746,4.155746,4.155679,4.045759,4.047768
1752346,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,2.800001e-07,4.250735,4.250744,362,23,4.250744,4.250744,4.250654,4.435799,4.438573
1752347,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,2.800000e-07,4.347929,4.347902,362,23,4.347902,4.347902,4.347777,5.532094,5.534868
1752348,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,2.800000e-07,4.438489,4.438525,362,23,4.438525,4.438525,4.438403,7.391329,7.391820


In [28]:
ice_data_df = pd.read_csv("./../../output/py_icesnow.csv")
ice_data_df = ice_data_df[1:] # considering everything from 2nd time step

num_time_steps = ice_data_df.shape[0]
depth_list = np.array(list(range(1, depth_steps+1)) * num_time_steps)
depth_df = pd.DataFrame(data={'depth':depth_list})

#repeating the dataframe depth_steps number of times
ice_data_df = pd.DataFrame(np.repeat(ice_data_df.values, depth_steps, axis=0), columns=ice_data_df.columns)
ice_data_df = pd.concat([depth_df, ice_data_df], ignore_index=False, axis=1)
print(ice_data_df)

final_df = final_df.merge(ice_data_df, how='inner', on=['time','depth'])
final_df

         depth                 time       ice      snow   snowice
0            1  2015-12-30 01:00:00       0.0       0.0       0.0
1            2  2015-12-30 01:00:00       0.0       0.0       0.0
2            3  2015-12-30 01:00:00       0.0       0.0       0.0
3            4  2015-12-30 01:00:00       0.0       0.0       0.0
4            5  2015-12-30 01:00:00       0.0       0.0       0.0
...        ...                  ...       ...       ...       ...
1751945     46  2019-12-28 23:00:00  0.283046  0.025199  0.044785
1751946     47  2019-12-28 23:00:00  0.283046  0.025199  0.044785
1751947     48  2019-12-28 23:00:00  0.283046  0.025199  0.044785
1751948     49  2019-12-28 23:00:00  0.283046  0.025199  0.044785
1751949     50  2019-12-28 23:00:00  0.283046  0.025199  0.044785

[1751950 rows x 5 columns]


,depth,time,AirTemp_degC,Longwave_Wm-2,Latent_Wm-2,Sensible_Wm-2,Shortwave_Wm-2,lightExtinct_m-1,ShearVelocity_mS-1,ShearStress_Nm-2,...,day_of_year,time_of_day,temp_mix03,temp_conv04,temp_initial00,obs_temp,input_obs,ice,snow,snowice
0,1,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,364,1,-0.115875,-0.115875,-0.019025,5.558093,5.560076,0.0,0.0,0.0
1,2,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,364,1,0.470220,0.470220,-0.019025,4.870203,4.876163,0.0,0.0,0.0
2,3,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,364,1,1.092054,1.073671,0.046866,5.106928,5.112209,0.0,0.0,0.0
3,4,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,364,1,1.054755,1.073671,0.799295,5.106928,5.112209,0.0,0.0,0.0
4,5,2015-12-30 01:00:00,-1.26053,565.10046,-4.184752,-4.714701,0.0,0.4,-999.0,-999.0,...,364,1,1.312011,1.312011,1.381341,5.106928,5.112209,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1752345,46,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,362,23,4.155746,4.155746,4.155679,4.045759,4.047768,0.283046,0.025199,0.044785
1752346,47,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,362,23,4.250744,4.250744,4.250654,4.435799,4.438573,0.283046,0.025199,0.044785
1752347,48,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,362,23,4.347902,4.347902,4.347777,5.532094,5.534868,0.283046,0.025199,0.044785
1752348,49,2019-12-28 23:00:00,-3.080809,529.16738,-15.814628,-20.288304,0.0,0.4,-999.0,-999.0,...,362,23,4.438525,4.438525,4.438403,7.391329,7.391820,0.283046,0.025199,0.044785


In [29]:
obs_array = final_df['obs_temp']
obs_array[obs_array == -999] = final_df['temp_total05']
print(obs_array)
final_df['obs_temp'] = obs_array

0           5.558093
1           4.870203
2           5.106928
3           5.106928
4           5.106928
             ...    
1752345     4.045759
1752346     4.435799
1752347     5.532094
1752348     7.391329
1752349    11.007965
Name: obs_temp, Length: 1752350, dtype: float64


C:\Users\ladwi\AppData\Local\Temp\ipykernel_13712\1668945599.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  obs_array[obs_array == -999] = final_df['temp_total05']


In [30]:
final_df_null = final_df.fillna('')
print(final_df_null.head)

<bound method NDFrame.head of          depth                 time  AirTemp_degC  Longwave_Wm-2  Latent_Wm-2  \
0            1  2015-12-30 01:00:00     -1.260530      565.10046    -4.184752   
1            2  2015-12-30 01:00:00     -1.260530      565.10046    -4.184752   
2            3  2015-12-30 01:00:00     -1.260530      565.10046    -4.184752   
3            4  2015-12-30 01:00:00     -1.260530      565.10046    -4.184752   
4            5  2015-12-30 01:00:00     -1.260530      565.10046    -4.184752   
...        ...                  ...           ...            ...          ...   
1752345     46  2019-12-28 23:00:00     -3.080809      529.16738   -15.814628   
1752346     47  2019-12-28 23:00:00     -3.080809      529.16738   -15.814628   
1752347     48  2019-12-28 23:00:00     -3.080809      529.16738   -15.814628   
1752348     49  2019-12-28 23:00:00     -3.080809      529.16738   -15.814628   
1752349     50  2019-12-28 23:00:00     -3.080809      529.16738   -15.814628  

In [31]:
final_df_null.to_csv("./../02_training/all_data_lake_modeling_in_time.csv", index=False)

In [32]:
final_df_null.to_csv("all_data_lake_modeling_in_time.csv", index=False)